In [483]:
import pandas as pd
from sqlalchemy import create_engine
import datetime as dt
import uuid

In [484]:
#Bring in the CSV files / data

csv_path = "Resources/covid_19_data.csv"
csv_path2 = "Resources/H1N1_2009.csv"

covid = pd.read_csv(csv_path, parse_dates=["ObservationDate"])
h1n1 = pd.read_csv(csv_path2, parse_dates=["Update Time"],encoding = 'unicode_escape')


In [485]:
covid
h1n1
covid

,SNo,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,1,2020-01-22,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,2,2020-01-22,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,3,2020-01-22,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,4,2020-01-22,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,5,2020-01-22,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
10666,10667,2020-03-31,Wyoming,US,2020-03-31 23:49:27,109.0,0.0,0.0
10667,10668,2020-03-31,Xinjiang,Mainland China,2020-03-31 23:49:27,76.0,3.0,73.0
10668,10669,2020-03-31,Yukon,Canada,2020-03-31 23:49:27,5.0,0.0,0.0
10669,10670,2020-03-31,Yunnan,Mainland China,2020-03-31 23:49:27,182.0,2.0,172.0


# COVID-19 Data Tables

In [486]:
covid = covid.loc[:,['ObservationDate', 'Province/State', 'Country/Region', 'Confirmed', 'Deaths', 'Recovered']]

#Rename Columns
covid = covid.rename(columns={"ObservationDate": "Date", "Country/Region" : "Country"})

#Replace Values for country naming consistency
replace_values = {"(St. Martin)" : "St. Martin", "('St. Martin',)": "St. Martin", 
                  'Republic of Ireland' : "Ireland", 'Cabo Verde' : "Cape Verde" } 

covid = covid.replace({"Country": replace_values})

covid["Country"].unique()

array(['Mainland China', 'Hong Kong', 'Macau', 'Taiwan', 'US', 'Japan',
       'Thailand', 'South Korea', 'Singapore', 'Philippines', 'Malaysia',
       'Vietnam', 'Australia', 'Mexico', 'Brazil', 'Colombia', 'France',
       'Nepal', 'Canada', 'Cambodia', 'Sri Lanka', 'Ivory Coast',
       'Germany', 'Finland', 'United Arab Emirates', 'India', 'Italy',
       'UK', 'Russia', 'Sweden', 'Spain', 'Belgium', 'Others', 'Egypt',
       'Iran', 'Israel', 'Lebanon', 'Iraq', 'Oman', 'Afghanistan',
       'Bahrain', 'Kuwait', 'Austria', 'Algeria', 'Croatia',
       'Switzerland', 'Pakistan', 'Georgia', 'Greece', 'North Macedonia',
       'Norway', 'Romania', 'Denmark', 'Estonia', 'Netherlands',
       'San Marino', ' Azerbaijan', 'Belarus', 'Iceland', 'Lithuania',
       'New Zealand', 'Nigeria', 'North Ireland', 'Ireland', 'Luxembourg',
       'Monaco', 'Qatar', 'Ecuador', 'Azerbaijan', 'Czech Republic',
       'Armenia', 'Dominican Republic', 'Indonesia', 'Portugal',
       'Andorra', 'Latvia

In [487]:
covid = covid[['Country', 'Province/State', 'Date', 'Confirmed', 'Deaths', 'Recovered']]
covid

,Country,Province/State,Date,Confirmed,Deaths,Recovered
0,Mainland China,Anhui,2020-01-22,1.0,0.0,0.0
1,Mainland China,Beijing,2020-01-22,14.0,0.0,0.0
2,Mainland China,Chongqing,2020-01-22,6.0,0.0,0.0
3,Mainland China,Fujian,2020-01-22,1.0,0.0,0.0
4,Mainland China,Gansu,2020-01-22,0.0,0.0,0.0
...,...,...,...,...,...,...
10666,US,Wyoming,2020-03-31,109.0,0.0,0.0
10667,Mainland China,Xinjiang,2020-03-31,76.0,3.0,73.0
10668,Canada,Yukon,2020-03-31,5.0,0.0,0.0
10669,Mainland China,Yunnan,2020-03-31,182.0,2.0,172.0


In [488]:
#Group Provinces and take largest cumulative confirmed and death number
province_df = covid.groupby(by='Province/State').agg('max').reset_index(drop=False)

#Group all provinces into their countries and add confirmed and death numbers
province_df = province_df.groupby(by='Country').agg('sum').reset_index(drop=False)

province_df

,Country,Confirmed,Deaths,Recovered
0,Australia,4559.0,18.0,358.0
1,Canada,8551.0,100.0,14.0
2,Denmark,1505.0,13.0,77.0
3,France,34824.0,1431.0,2281.0
4,Germany,5.0,0.0,0.0
5,Hong Kong,714.0,4.0,128.0
6,Israel,8.0,0.0,0.0
7,Lebanon,2.0,0.0,0.0
8,Macau,41.0,0.0,10.0
9,Mainland China,81525.0,3305.0,76068.0


In [489]:
#Remove countries that are in province_df dataset
remove_list = province_df['Country']
global_covid_data = covid[~covid['Country'].isin(remove_list)]

#province_df
global_covid_data = global_covid_data.loc[:,['Country', 'Date', 'Confirmed', 'Deaths', 'Recovered']]
global_covid_data = global_covid_data.groupby(by='Country').agg('sum').reset_index(drop=False)


global_covid_data

,Country,Confirmed,Deaths,Recovered
0,Azerbaijan,1.0,0.0,0.0
1,Afghanistan,1225.0,29.0,26.0
2,Albania,2232.0,91.0,241.0
3,Algeria,4828.0,324.0,594.0
4,Andorra,2787.0,39.0,37.0
...,...,...,...,...
192,Vietnam,2930.0,0.0,818.0
193,West Bank and Gaza,617.0,6.0,106.0
194,Zambia,194.0,0.0,0.0
195,Zimbabwe,53.0,9.0,0.0


In [490]:
#Merge province and country data
global_covid_data = pd.concat([global_covid_data, province_df], ignore_index=True)

In [491]:
#global_covid_data consists of every country with affected persons - total number of; Confirmed, Deaths, Recovered

global_covid_data

,Country,Confirmed,Deaths,Recovered
0,Azerbaijan,1.0,0.0,0.0
1,Afghanistan,1225.0,29.0,26.0
2,Albania,2232.0,91.0,241.0
3,Algeria,4828.0,324.0,594.0
4,Andorra,2787.0,39.0,37.0
...,...,...,...,...
207,Netherlands,3703.0,137.0,5.0
208,Others,61.0,0.0,0.0
209,Taiwan,47.0,1.0,17.0
210,UK,5805.0,245.0,127.0


# H1N1 Data Tables

In [492]:
global_h1n1_data = h1n1.groupby(by='Country').agg('max').reset_index(drop=False)

#Rename Columns
global_h1n1_data = global_h1n1_data.rename(columns={"Cases": "Confirmed", "Update Time": "Date"})

global_h1n1_data


,Country,Confirmed,Deaths,Date
0,Algeria,5,0.0,2009-07-06 09:00:00
1,Antigua and Barbuda,2,0.0,2009-07-06 09:00:00
2,Argentina,2485,60.0,2009-07-06 09:00:00
3,Australia,5298,10.0,2009-07-06 09:00:00
4,Austria,19,0.0,2009-07-06 09:00:00
...,...,...,...,...
142,"Netherlands Antilles, Curaçao",8,0.0,2009-07-06 09:00:00
143,"Netherlands Antilles, Curaçao *",3,0.0,2009-06-29 09:00:00
144,"Netherlands Antilles, Curaçao **",3,0.0,2009-06-22 07:00:00
145,"Netherlands Antilles, Sint Maarten",7,0.0,2009-07-06 09:00:00


In [493]:
h1n1 = h1n1.rename(columns={"Cases": "Confirmed", "Update Time": "Date"})

h1n1

,Country,Confirmed,Deaths,Date
0,Yemen,8,0.0,2009-07-06 09:00:00
1,Yemen,7,0.0,2009-07-03 09:00:00
2,Yemen,7,0.0,2009-07-01 09:00:00
3,Yemen,6,0.0,2009-06-29 09:00:00
4,Yemen,6,0.0,2009-06-26 07:00:00
...,...,...,...,...
1817,"Guernsey, Crown Dependency",5,0.0,2009-07-06 09:00:00
1818,"Guernsey, Crown Dependency",5,0.0,2009-07-03 09:00:00
1819,"Guernsey, Crown Dependency",5,0.0,2009-07-01 09:00:00
1820,"Guernsey, Crown Dependency",1,0.0,2009-06-29 09:00:00


In [494]:
#Create lists of all countries
country_covid = global_covid_data['Country']
country_h1n1 = global_h1n1_data['Country']

#Combine country lists together
country_df = pd.concat([country_covid, country_h1n1], ignore_index=True)

#Put countries into a DataFrame
country_df = pd.DataFrame(country_df)

#Drop Duplicate Countries
country_df = country_df.drop_duplicates("Country")

#Reset Index and make new index as a column
country_df = country_df.reset_index(drop=True)
country_df = country_df.reset_index(level=0)

#Rename index column to Country ID
country_df = country_df.rename(columns={"index": "Country ID"})

country_df = country_df[['Country', 'Country ID']]

country_df



,Country,Country ID
0,Azerbaijan,0
1,Afghanistan,1
2,Albania,2
3,Algeria,3
4,Andorra,4
...,...,...
235,"Netherlands Antilles, Curaçao",235
236,"Netherlands Antilles, Curaçao *",236
237,"Netherlands Antilles, Curaçao **",237
238,"Netherlands Antilles, Sint Maarten",238


In [495]:
#Merge on global_covid_data
global_covid_data = pd.merge(global_covid_data, country_df, how='inner', on='Country')
global_covid_data = global_covid_data[['Country ID', 'Country', 'Confirmed', 'Deaths', 'Recovered']]

In [496]:
#Merge on global_h1n1_data
global_h1n1_data = pd.merge(global_h1n1_data, country_df, how='inner', on='Country')
global_h1n1_data = global_h1n1_data[['Country ID', 'Country', 'Confirmed', 'Deaths']]
global_h1n1_data

,Country ID,Country,Confirmed,Deaths
0,3,Algeria,5,0.0
1,6,Antigua and Barbuda,2,0.0
2,7,Argentina,2485,60.0
3,197,Australia,5298,10.0
4,10,Austria,19,0.0
...,...,...,...,...
142,235,"Netherlands Antilles, Curaçao",8,0.0
143,236,"Netherlands Antilles, Curaçao *",3,0.0
144,237,"Netherlands Antilles, Curaçao **",3,0.0
145,238,"Netherlands Antilles, Sint Maarten",7,0.0


In [497]:
global_covid_data

,Country ID,Country,Confirmed,Deaths,Recovered
0,0,Azerbaijan,1.0,0.0,0.0
1,1,Afghanistan,1225.0,29.0,26.0
2,2,Albania,2232.0,91.0,241.0
3,3,Algeria,4828.0,324.0,594.0
4,4,Andorra,2787.0,39.0,37.0
...,...,...,...,...,...
207,207,Netherlands,3703.0,137.0,5.0
208,208,Others,61.0,0.0,0.0
209,209,Taiwan,47.0,1.0,17.0
210,210,UK,5805.0,245.0,127.0


In [498]:
country_df.head(20)

,Country,Country ID
0,Azerbaijan,0
1,Afghanistan,1
2,Albania,2
3,Algeria,3
4,Andorra,4
5,Angola,5
6,Antigua and Barbuda,6
7,Argentina,7
8,Armenia,8
9,Aruba,9


In [499]:
#Merge on covid
covid = pd.merge(covid, country_df, how='inner', on='Country')
covid = covid[['Country ID', 'Country', 'Province/State', 'Date', 'Confirmed', 'Deaths', 'Recovered']]
covid

,Country ID,Country,Province/State,Date,Confirmed,Deaths,Recovered
0,206,Mainland China,Anhui,2020-01-22,1.0,0.0,0.0
1,206,Mainland China,Beijing,2020-01-22,14.0,0.0,0.0
2,206,Mainland China,Chongqing,2020-01-22,6.0,0.0,0.0
3,206,Mainland China,Fujian,2020-01-22,1.0,0.0,0.0
4,206,Mainland China,Gansu,2020-01-22,0.0,0.0,0.0
...,...,...,...,...,...,...,...
10666,109,MS Zaandam,NaN,2020-03-31,2.0,0.0,0.0
10667,24,Botswana,NaN,2020-03-30,3.0,0.0,0.0
10668,24,Botswana,NaN,2020-03-31,4.0,1.0,0.0
10669,30,Burundi,NaN,2020-03-31,2.0,0.0,0.0


In [500]:
#Merge on h1n1
h1n1 = pd.merge(h1n1, country_df, how='inner', on='Country')
h1n1 = h1n1[['Country ID', 'Country', 'Date', 'Confirmed', 'Deaths']]
h1n1

,Country ID,Country,Date,Confirmed,Deaths
0,228,Yemen,2009-07-06 09:00:00,8,0.0
1,228,Yemen,2009-07-03 09:00:00,7,0.0
2,228,Yemen,2009-07-01 09:00:00,7,0.0
3,228,Yemen,2009-06-29 09:00:00,6,0.0
4,228,Yemen,2009-06-26 07:00:00,6,0.0
...,...,...,...,...,...
1817,232,"Guernsey, Crown Dependency",2009-07-06 09:00:00,5,0.0
1818,232,"Guernsey, Crown Dependency",2009-07-03 09:00:00,5,0.0
1819,232,"Guernsey, Crown Dependency",2009-07-01 09:00:00,5,0.0
1820,232,"Guernsey, Crown Dependency",2009-06-29 09:00:00,1,0.0


In [501]:
#Set Index for all dataframes
country_df.set_index("Country ID", inplace=True)
global_covid_data.set_index("Country ID", inplace=True)
global_h1n1_data.set_index("Country ID", inplace=True)
covid.set_index("Country ID", inplace=True)
h1n1.set_index("Country ID", inplace=True)


In [502]:
global_covid_data

,Country,Confirmed,Deaths,Recovered
Country ID,,,,
0,Azerbaijan,1.0,0.0,0.0
1,Afghanistan,1225.0,29.0,26.0
2,Albania,2232.0,91.0,241.0
3,Algeria,4828.0,324.0,594.0
4,Andorra,2787.0,39.0,37.0
...,...,...,...,...
207,Netherlands,3703.0,137.0,5.0
208,Others,61.0,0.0,0.0
209,Taiwan,47.0,1.0,17.0


In [503]:
global_h1n1_data

,Country,Confirmed,Deaths
Country ID,,,
3,Algeria,5,0.0
6,Antigua and Barbuda,2,0.0
7,Argentina,2485,60.0
197,Australia,5298,10.0
10,Austria,19,0.0
...,...,...,...
235,"Netherlands Antilles, Curaçao",8,0.0
236,"Netherlands Antilles, Curaçao *",3,0.0
237,"Netherlands Antilles, Curaçao **",3,0.0


In [504]:
covid

,Country,Province/State,Date,Confirmed,Deaths,Recovered
Country ID,,,,,,
206,Mainland China,Anhui,2020-01-22,1.0,0.0,0.0
206,Mainland China,Beijing,2020-01-22,14.0,0.0,0.0
206,Mainland China,Chongqing,2020-01-22,6.0,0.0,0.0
206,Mainland China,Fujian,2020-01-22,1.0,0.0,0.0
206,Mainland China,Gansu,2020-01-22,0.0,0.0,0.0
...,...,...,...,...,...,...
109,MS Zaandam,NaN,2020-03-31,2.0,0.0,0.0
24,Botswana,NaN,2020-03-30,3.0,0.0,0.0
24,Botswana,NaN,2020-03-31,4.0,1.0,0.0


In [505]:
h1n1

,Country,Date,Confirmed,Deaths
Country ID,,,,
228,Yemen,2009-07-06 09:00:00,8,0.0
228,Yemen,2009-07-03 09:00:00,7,0.0
228,Yemen,2009-07-01 09:00:00,7,0.0
228,Yemen,2009-06-29 09:00:00,6,0.0
228,Yemen,2009-06-26 07:00:00,6,0.0
...,...,...,...,...
232,"Guernsey, Crown Dependency",2009-07-06 09:00:00,5,0.0
232,"Guernsey, Crown Dependency",2009-07-03 09:00:00,5,0.0
232,"Guernsey, Crown Dependency",2009-07-01 09:00:00,5,0.0


In [506]:
connection_string = "postgres:postgres@localhost:5432/COVID19_vs_H1N1"
engine = create_engine(f'postgresql://{connection_string}')

In [507]:
engine.table_names()

['global_covid_data', 'global_h1n1_data', 'covid', 'h1n1', 'country']

# Load DataFrames into database

In [508]:
global_covid_data.to_sql(name='global_covid_data', con=engine, if_exists='append', index=True)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "Country ID" of relation "global_covid_data" does not exist
LINE 1: INSERT INTO global_covid_data ("Country ID", "Country", "Con...
                                       ^

[SQL: INSERT INTO global_covid_data ("Country ID", "Country", "Confirmed", "Deaths", "Recovered") VALUES (%(Country ID)s, %(Country)s, %(Confirmed)s, %(Deaths)s, %(Recovered)s)]
[parameters: ({'Country ID': 0, 'Country': ' Azerbaijan', 'Confirmed': 1.0, 'Deaths': 0.0, 'Recovered': 0.0}, {'Country ID': 1, 'Country': 'Afghanistan', 'Confirmed': 1225.0, 'Deaths': 29.0, 'Recovered': 26.0}, {'Country ID': 2, 'Country': 'Albania', 'Confirmed': 2232.0, 'Deaths': 91.0, 'Recovered': 241.0}, {'Country ID': 3, 'Country': 'Algeria', 'Confirmed': 4828.0, 'Deaths': 324.0, 'Recovered': 594.0}, {'Country ID': 4, 'Country': 'Andorra', 'Confirmed': 2787.0, 'Deaths': 39.0, 'Recovered': 37.0}, {'Country ID': 5, 'Country': 'Angola', 'Confirmed': 48.0, 'Deaths': 6.0, 'Recovered'